In [2]:
import numpy as np 
import pandas as pd
import seaborn as sns

# Question 1 

For this question, you only need to do programming for Part 1.1 - 1.6. You will only be asked to provide answers in Part 1.7. 

In [34]:
PATH_TO_Q1_DATA = 'data/HW3_Q1_DATA_1000.csv' #TODO: Change if your path to data is different
df_original = pd.read_csv(PATH_TO_Q1_DATA)


### Data Exploration

In [35]:
df_original.shape

(998, 19)

In [36]:
df_original.head()

,Sex,Intubated,Pneumonia,Age,Pregnant,Diabetes,COPD,Asthma,Immunocompromised,Hypertension,Other_Disease,Cardiovascular_disease,Obesity,Renal_disease,Smoker,Exposure_to_others_with_COVID,Has_COVID,ICU,Died
0,F,N,N,54,N,N,N,N,N,N,N,N,Y,N,N,NaN,Y,N,N
1,M,N,Y,30,NaN,N,N,N,N,N,N,N,N,N,N,NaN,Y,N,N
2,F,N,N,60,N,Y,N,N,N,Y,N,Y,N,N,N,NaN,Y,N,Y
3,M,N,Y,47,NaN,Y,N,N,N,N,N,N,N,N,N,NaN,Y,Y,Y
4,M,N,N,63,NaN,N,N,N,N,Y,N,N,N,N,N,NaN,Y,N,N


In [37]:
# convert categorical data to numerical values
# drop one of each of the binary categories because it is not necessary (all info is encoded by one column)
# add column for missing data
df=pd.get_dummies(df_original, drop_first=True, dummy_na=True)

In [38]:
df.head()

,Age,Sex_M,Sex_nan,Intubated_Y,Intubated_nan,Pneumonia_Y,Pneumonia_nan,Pregnant_Y,Pregnant_nan,Diabetes_Y,...,Renal_disease_nan,Smoker_Y,Smoker_nan,Exposure_to_others_with_COVID_Y,Exposure_to_others_with_COVID_nan,Has_COVID_nan,ICU_Y,ICU_nan,Died_Y,Died_nan
0,54,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,30,1,0,0,0,1,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
2,60,0,0,0,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,1,0
3,47,1,0,0,0,1,0,0,1,1,...,0,0,0,0,1,0,1,0,1,0
4,63,1,0,0,0,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0


In [39]:
df.sum(axis='rows')

Age                                  55014
Sex_M                                  616
Sex_nan                                  0
Intubated_Y                             23
Intubated_nan                            1
Pneumonia_Y                            525
Pneumonia_nan                            0
Pregnant_Y                               2
Pregnant_nan                           616
Diabetes_Y                             313
Diabetes_nan                             2
COPD_Y                                  37
COPD_nan                                 1
Asthma_Y                                22
Asthma_nan                               2
Immunocompromised_Y                     33
Immunocompromised_nan                    1
Hypertension_Y                         354
Hypertension_nan                         1
Other_Disease_Y                         66
Other_Disease_nan                        1
Cardiovascular_disease_Y                47
Cardiovascular_disease_nan               1
Obesity_Y  

#### Separate dataset into Data and Target

In [40]:
y = df.loc[:]['Died_Y']
print(y.shape)
print(y.name)
X = df.iloc[:,:-2]  # remove last two columns
print(X.shape)
print(list(X.columns))

(998,)
Died_Y
(998, 34)
['Age', 'Sex_M', 'Sex_nan', 'Intubated_Y', 'Intubated_nan', 'Pneumonia_Y', 'Pneumonia_nan', 'Pregnant_Y', 'Pregnant_nan', 'Diabetes_Y', 'Diabetes_nan', 'COPD_Y', 'COPD_nan', 'Asthma_Y', 'Asthma_nan', 'Immunocompromised_Y', 'Immunocompromised_nan', 'Hypertension_Y', 'Hypertension_nan', 'Other_Disease_Y', 'Other_Disease_nan', 'Cardiovascular_disease_Y', 'Cardiovascular_disease_nan', 'Obesity_Y', 'Obesity_nan', 'Renal_disease_Y', 'Renal_disease_nan', 'Smoker_Y', 'Smoker_nan', 'Exposure_to_others_with_COVID_Y', 'Exposure_to_others_with_COVID_nan', 'Has_COVID_nan', 'ICU_Y', 'ICU_nan']


## Part 1.1 Select features and train classifiers

### Part 1.1 Work 

In [41]:
# TODO: 
#   Step 1: Select and apply a filter-based or wrapper-based feature selection method to the data.
#   Step 2: Train a classifier using the selected features. Use 10-fold cross validation.

# Tip: 
#   1. You may find the filter-based or wrapper-based methods you used in HW2 useful. 

#   2. Scikit-learn implement many classifiers, see the comparisons of their 
#   performance and introductions here:
#   https://scikit-learn.org/stable/auto_examples/classification/plot_classifier_comparison.html

#   3. Scikit-learn also implement classifiers with built-in cross validations,
#   for example: https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeClassifierCV.html

#### Select features using wrapper-based feature selection

In [42]:
%%time 
# runtime - 289 ms (10 records, 2 fold) 
# runtime - 13.3 s (1000 records, 10 fold)
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import RFECV

svc = SVC(kernel="linear")
rfecv = RFECV(estimator=svc, step=1, cv=StratifiedKFold(10),scoring='accuracy')
rfecv.fit(X, y)

CPU times: user 51.7 s, sys: 349 ms, total: 52.1 s
Wall time: 13.3 s


RFECV(cv=StratifiedKFold(n_splits=10, random_state=None, shuffle=False),
      estimator=SVC(kernel='linear'), scoring='accuracy')

In [43]:
rfecv.support_

array([ True,  True, False,  True,  True,  True, False, False,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True, False,  True,  True])

In [44]:
# select features which matter
list_selected_wrapper=list()
for i,val in enumerate(rfecv.support_):
    if val:
        list_selected_wrapper.append(X.columns[i])

In [45]:
list_selected_wrapper

['Age',
 'Sex_M',
 'Intubated_Y',
 'Intubated_nan',
 'Pneumonia_Y',
 'Pregnant_nan',
 'Diabetes_Y',
 'Diabetes_nan',
 'COPD_Y',
 'COPD_nan',
 'Asthma_Y',
 'Asthma_nan',
 'Immunocompromised_Y',
 'Immunocompromised_nan',
 'Hypertension_Y',
 'Hypertension_nan',
 'Other_Disease_Y',
 'Other_Disease_nan',
 'Cardiovascular_disease_Y',
 'Cardiovascular_disease_nan',
 'Obesity_Y',
 'Obesity_nan',
 'Renal_disease_Y',
 'Renal_disease_nan',
 'Smoker_Y',
 'Smoker_nan',
 'Exposure_to_others_with_COVID_Y',
 'Exposure_to_others_with_COVID_nan',
 'ICU_Y',
 'ICU_nan']

#### Train classifier

In [30]:
X_selected_wrapper = X.loc[:][list_selected_wrapper]
X_selected_wrapper.shape

(9, 2)

In [31]:
X_selected_wrapper.head()

,Diabetes_Y,ICU_Y
0,0,0
1,0,0
2,1,0
3,1,1
4,0,0


### Part 1.2 Work 

In [ ]:
# TODO: 
#   Step 1: Select a learning algorithm that performs embedded feature selection. 
#   Step 2: Train a classifier using the selected features. Use 10-fold cross validation.

# Tip: 
#   1. Scikit-learn implement many classifiers, see the comparisons of their 
#   performance and introductions here:
#   https://scikit-learn.org/stable/auto_examples/classification/plot_classifier_comparison.html

#   2. Scikit-learn also implement classifiers with built-in cross validations,
#   for example: https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeClassifierCV.html

### Part 1.3 Work 

In [ ]:
# TODO: 
#   Step 1: Select and apply a data imputation method to handle the missing data.
#   Step 2: Apply the  wrapper based feature selection method you used in part 1.1.
#   Step 3: Train a classifier using the selected features. Use the same classifier you used in part 1.1. Use 10-fold cross validation.

# Tip: 
#   1. Sciki-learn implements different imputation methods. Take a look at https://scikit-learn.org/stable/modules/impute.html
#   and use the one you think most appropriate.

### Part 1.4 Work 

In [ ]:
# TODO: 
#   Step 1: Apply a data imputation method to eliminate any missing values in the data. Use the same method you used in part 1.3. 
#   Step 2: Train a classifier. Use the same classifier you used in part 1.2. Use 10-fold cross validation.

### Part 1.5 Work 

In [ ]:
# TODO: 
#   Step 1: Apply a data imputation method to eliminate any missing values in the data. Use the same method you used in parts 1.3 & 1.4. 
#   Step 2: Select a learning algorithm that performs cost-sensitive learning. 
#   Step 3: Adjust the costs until you find a classifier that maximizes the F1-score, subject to the constraint that it achieves 95% sensitivity for the label ‘Y’.  Use 10-fold cross validation.

# Tip: 
#   1. F1-score: https://en.wikipedia.org/wiki/F1_score
#   2. Scikit-learn supports extending the classifiers to cost-sensitive learning.
#   Take a look at this tutorial: https://machinelearningmastery.com/cost-sensitive-learning-for-imbalanced-classification/

### Part 1.6 Work 

In [ ]:
# TODO: 
#   Step 1: Implement a function calculating the weighted average F1-score, following 
#       the steps in the homework problem statement.
#   Step 2: Find a classifier that achieves a weighted average F1-score of at least 0.74 using 10-fold cross validation.

# Tip: 
#   1. F1-score: https://en.wikipedia.org/wiki/F1_score
#   2. Scikit-learn supports extending the classifiers to cost-sensitive learning.
#   Take a look at this tutorial: https://machinelearningmastery.com/cost-sensitive-learning-for-imbalanced-classification/

### Part 1.7 Work

In [ ]:
# Tip:
#   1. Scikit-learn implements different evaluation metrics for classifications,
#   see: https://scikit-learn.org/stable/modules/model_evaluation.html

### Part 1.7 Answers

1. Create a ROC plot with the results from parts 1.1 to 1.6. 
2. Create a table with the following performance metrics for the results from parts 1.1 to 1.6:
    * Accuracy
    * Sensitivity & Specificity
    * The positive and negative predictive values
    * F1-score
    * The Matthews Correlation Coefficient
    * AUC (Area under the ROC curve)